In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

def fullRead(pathToTable, sep, anthro = False):

  df_renamed = pd.read_csv(pathToTable, sep = sep, encoding = "latin_1")


  # reading and merging    
  
  if anthro == True:
    df_anthro = pd.read_csv("data/chronicAnthropometricCardiovascularData.csv", sep=";", decimal=",")
    df_renamed = df_renamed.merge(df_anthro)

    # separating by time moment and renaming

    df_renamed["Weight"] = ""
    df_renamed["BMI"] = ""
    df_renamed["Fat"] = ""
    df_renamed["CVRI"] = ""
    df_renamed["Bpmin"] = ""
    df_renamed["Bpmax"] = ""
    df_renamed["Frec"] = ""

    for i in range(len(df_renamed)):

        if df_renamed.loc[i]["Time"] == "Initial":
            df_renamed.loc[i,"Weight"] = df_renamed.loc[i]["Peso inicial"]
            df_renamed.loc[i,"BMI"] = df_renamed.loc[i]["IMC Inicial"]
            df_renamed.loc[i,"Fat"] = df_renamed.loc[i]["Grasa inicial"]
            df_renamed.loc[i,"CVRI"] = df_renamed.loc[i]["IRCV inicial"] 
            df_renamed.loc[i,"Bpmin"] = df_renamed.loc[i]["Bpmin inicial"] 
            df_renamed.loc[i,"Bpmax"] = df_renamed.loc[i]["Bpmax inicial"] 
            df_renamed.loc[i,"Frec"] = df_renamed.loc[i]["Frec inicial"] 
                
        if df_renamed.loc[i]["Time"] == "Final":
        
            df_renamed.loc[i,"Weight"] = df_renamed.loc[i]["Peso final"]
            df_renamed.loc[i,"BMI"] = df_renamed.loc[i]["IMC Final"]
            df_renamed.loc[i,"Fat"] = df_renamed.loc[i]["Grasa final"]
            df_renamed.loc[i,"CVRI"] = df_renamed.loc[i]["IRCV Final"] 
            df_renamed.loc[i,"Bpmin"] = df_renamed.loc[i]["Bpmin final"] 
            df_renamed.loc[i,"Bpmax"] = df_renamed.loc[i]["Bpmax final"] 
            df_renamed.loc[i,"Frec"] = df_renamed.loc[i]["Frec final"] 
        
    df_renamed.drop(columns = ["Peso inicial", "Peso final", "Delta Peso", "Talla", "IMC Inicial", "IMC Final", "Delta IMC", "Grasa inicial", "Grasa final", "Delta Grasa", "IRCV Final", "IRCV inicial", "Bpmin final", "Bpmin inicial", "Bpmax final", "Bpmax inicial", "Frec final", "Frec inicial",], inplace=True )
  
  df_renamed.drop(columns = ["Unnamed: 0", "grouping"], inplace=True )
  df_renamed.fillna(0, inplace=True)
  return df_renamed

def scaling(df_read):
   
   scaler = preprocessing.MinMaxScaler()
   numCols = df_read.select_dtypes(include=np.number).drop("numVol",1).columns
   df_read[numCols] = scaler.fit_transform(df_read[numCols])
   return df_read

df1 = scaling(fullRead("data/urineAnt_ord.csv",  sep = ",", anthro= True))
df1.columns

C:\Users\dres2\AppData\Local\Temp\ipykernel_13960\2456273508.py:56: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  numCols = df_read.select_dtypes(include=np.number).drop("numVol",1).columns


Index(['numVol', 'CA', 'CA.G', 'CA.S', 'CA.GS', 'Total.CA', 'DHPAA', 'DHPAA.G',
       'DHPAA.GG', 'DHPAA.GS', 'DHPAA.SS', 'Total.DHPAA', 'TFA.G', 'TFA.S',
       'TFA.di.sulfate.1', 'Total.TFA', 'TIFA.Sulfate.1', 'VA', 'VA.GG',
       'VA.GS', 'VA.SS', 'Total.VA', 'Sweetener', 'Time', 'Sex', 'Weight',
       'BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec'],
      dtype='object')

Testing new slicing

In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

metabs = ['CA', 'CA.G', 'CA.S', 'CA.GS', 'Total.CA', 'DHPAA', 'DHPAA.G',
       'DHPAA.GG', 'DHPAA.GS', 'DHPAA.SS', 'Total.DHPAA', 'TFA.G', 'TFA.S',
       'TFA.di.sulfate.1', 'Total.TFA', 'TIFA.Sulfate.1', 'VA', 'VA.GG',
       'VA.GS', 'VA.SS', 'Total.VA']
anthro = ['Weight','BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec']
factors = ['Sweetener', 'Time', 'Sex']
enc = OrdinalEncoder()
enc.fit(df1[factors])
df1[factors] = enc.transform(df1[factors])

#df_0 = df1_anthro[df1_anthro["Time"] == 1]
#df_metab0 = df_0[metabs]

#df_metab0antrofull = df_metab0.join(df1_anthro[anthro + factors + ["numVol"]], )
#df_metab0antrofull["Time"]
X, y = df1.drop(["numVol"], axis=1), df1.drop(anthro + factors + ["numVol"], axis =1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.models import Sequential
from keras.layers import Dense
 
#enc = OrdinalEncoder()
#enc.fit(df1[["Sweetener", "Sex"]])
#df1[["Sweetener", "Sex"]] = enc.transform(df1[["Sweetener", "Sex"]])

#X, y = df1[df1["Time"] == "Initial"].drop(["numVol", "Time"], axis=1), df1[df1["Time"] == "Final"].drop(['Time','numVol', 'Sweetener', 'Time', 'Sex', 'Weight','BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec'], axis = 1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# get the model
def get_model(n_inputs, n_outputs):
 model = Sequential()
 model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
 model.add(Dense(n_outputs))
 model.compile(loss='mae', optimizer='adam')
 return model

# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y, X_train, X_test, y_train, y_test):
 results = list()
 n_inputs, n_outputs = X.shape[1], y.shape[1]
 # define evaluation procedure
 cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
 # enumerate folds
 #for train_ix, test_ix in cv.split(X):
  #  print (train_ix)
    # prepare data
    # X_train, X_test = X[train_ix], X[test_ix]
    # y_train, y_test = y[train_ix], y[test_ix]
    # define model
 model = get_model(n_inputs, n_outputs)
 # fit model
 model.fit(X_train, y_train, verbose=0, epochs=500)
 # evaluate model on test set
 mae = model.evaluate(X_test, y_test, verbose=0)
 # store result
 print('>%.3f' % mae)
 results.append(mae)
 return results 

# evaluate model
results = evaluate_model(X, y, X_train, X_test, y_train, y_test)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

>0.074
MAE: 0.074 (0.000)


In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

X, y = df1[df1["Time"] == "Initial"].drop(["numVol", "Time"], axis=1), df1[df1["Time"] == "Final"].drop(['Time','numVol', 'Sweetener', 'Time', 'Sex', 'Weight','BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define custom feature weights
#weights = [0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.043, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]

# Create the deep learning model
model = Sequential()
model.add(Dense(64, input_shape=(X.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='linear'))

# Compile the model with weighted loss function
def weighted_loss(weights):
    def loss(y_true, y_pred):
        return np.sum(np.square((y_true - y_pred) * weights))
    return loss

model.compile(loss="mae", optimizer='adam')

# Fit the model to the training data
model.fit(X_train, y_train, epochs=500, batch_size=100, validation_data=(X_test, y_test), verbose = 0)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
score = model.evaluate(X_test, y_test)
score

NameError: name 'train_test_split' is not defined

In [5]:
X, y = df1[df1["Time"] == "Initial"].drop(["numVol", "Time"], axis=1), df1[df1["Time"] == "Final"].drop(['Time','numVol', 'Sweetener', 'Time', 'Sex', 'Weight','BMI', 'Fat', 'CVRI', 'Bpmin', 'Bpmax', 'Frec'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define custom weights for input features
#feature_weights = np.array([0.2, 0.5, 0.8, 1.0, 0.7, 0.5, 0.6, 0.9, 0.8, 0.5, 
#                            0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 0.6, 0.4, 0.3, 0.5, 
#                            0.8, 0.9, 1.0, 0.8, 0.7, 0.5, 0.4, 0.3, 0.2, 0.1])

# Initialize XGBoost model with multioutput regression objective
model = xgb.XGBRegressor(objective='reg:squarederror')

# Define the hyperparameter grid to search over
param_grid = {'n_estimators': [100, 500, 1000],
              'learning_rate': [0.01, 0.1, 0.5],
              'max_depth': [3, 5, 7]}

# Define the cross-validation scheme
cv = KFold(n_splits=10, shuffle=True, random_state=42)

# Define the scoring metrics to use
scoring = {'mse': 'neg_mean_squared_error',
           'mae': 'neg_mean_absolute_error',
           'r2': 'r2'}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=scoring, refit='mse')
grid_search.fit(X_train, y_train) #, sample_weight=feature_weights)

# Print the best hyperparameters and corresponding performance metrics
print('Best hyperparameters:', grid_search.best_params_)
print('Best mean squared error:', -grid_search.best_score_)
print('Best mean absolute error:', -grid_search.cv_results_['mean_test_mae'][grid_search.best_index_])
print('Best R-squared:', grid_search.cv_results_['mean_test_r2'][grid_search.best_index_])

# Use the best model to simulate an interventional study
# Assume you have loaded the test data into a separate array called X_test
y_pred = grid_search.best_estimator_.predict(X_test)

# y_pred is a 2D array with 21 columns, one for each response variable
# Each row corresponds to a single patient in the test set


ValueError: 
All the 270 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py", line 436, in inner_f
    return f(**kwargs)
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py", line 711, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py", line 236, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py", 

In [8]:
from sklearn.model_selection import GridSearchCV, KFold, RandomizedSearchCV

params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

xgbr = xgb.XGBRegressor(seed = 20)

clf = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=1)

clf.fit(X_train, y_train)
print("Best parameters:", clf.best_params_)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))

Fitting 5 folds for each of 25 candidates, totalling 125 fits


ValueError: 
All the 125 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py", line 436, in inner_f
    return f(**kwargs)
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py", line 711, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py", line 236, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  File "c:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py", 